In [1]:
from datetime import datetime
#from google.colab import drive
#drive.mount('/content/drive')
from datetime import timedelta
import pandas as pd
import nltk
import matplotlib.pyplot as plt
from nltk import word_tokenize
import requests
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import statsmodels.api as sm


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Sentiment Vader Anlaysis

In [2]:
def clean_tweet(sentence):
    words = word_tokenize(sentence)
    lowercase_words = [word.lower() for word in words
                       if word not in stopwords.words() and word.isalpha()]
    temp_text = ' '.join(lowercase_words)
    return temp_text

In [3]:
def vader_comparison(sentence):
    sentence=clean_tweet(sentence)
    analyzer = SentimentIntensityAnalyzer()
    pos=compound=neu=neg=0
    vs = analyzer.polarity_scores(sentence)
    pos = vs['pos']
    neu = vs['neu']
    neg = vs['neg']
    compound = vs['compound']
    if (compound > 0):
        return 1
    elif (compound == 0):
        return 0
    else:
        return -1

Function to run VADER sentiment analysis

In [11]:
def process_chunk(chunk):
    for index, row in chunk.iterrows():
        try:
            text = str(row['text']) if pd.notna(row['text']) else ''
            # Apply sentiment analysis to the text in the current row
            sentiment_score = vader_comparison(row['text'])
            # Store the sentiment score in a new column
            chunk.at[index, 'sentiment_vader'] = sentiment_score
        except Exception as e:
            print(f"Error processing row {index}: {e}")
            print("Text that caused the error:", row['text'])
    return chunk


Function to read the dataset in chunks as the Tweet dataset is huge

In [9]:
# Function to read the dataset in chunks and process each chunk
def process_dataset_in_chunks(dataset_path,  start_chunk_index, chunk_size=1000):
    chunk_iterator = pd.read_csv(dataset_path, chunksize=chunk_size, lineterminator = '\n')
    for i, chunk in enumerate(chunk_iterator):
        if i < start_chunk_index:
            continue  # Skip chunks until reaching the start_chunk_index

        print(f"Processing chunk {i+1}")
        processed_chunk = process_chunk(chunk)
        output_file = fr"C:\Users\user\OneDrive\Desktop\Research Project\Chunk Folder\processed_chunk_{i+1}.csv"
        processed_chunk.to_csv(output_file, index=False)
        print(f"Chunk {i+1} processed and saved to {output_file}")


In [4]:
dataset_path = r"C:\Users\user\OneDrive\Desktop\Research Project\Codes\bitcoin-tweets-2021.csv"

In [ ]:
start_chunk_index = 8000
process_dataset_in_chunks(dataset_path, start_chunk_index)